<a href="https://colab.research.google.com/github/IAT-ComputationalCreativity-Spring2025/Week11-Genetic-Algorithms/blob/main/DEAP_image_evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Evolution with DEAP

## Setup and Imports

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import random
import time

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Display settings for better visualization
plt.rcParams['figure.figsize'] = (12, 6)

## Create Target Image Function

Let's generate our target image as a size and shape

In [ ]:
def create_target_image(size=50, shape="circle"):
    """
    Create a target image for evolution to match.
    
    Parameters:
    - size: Size of the square image in pixels
    - shape: Type of shape ("circle", "square", "triangle", or "cross")
    
    Returns:
    - 2D numpy array representing the image
    """
    image = np.zeros((size, size))
    center = size // 2
    radius = size // 4
    
    if shape == "circle":
        for i in range(size):
            for j in range(size):
                if (i - center)**2 + (j - center)**2 < radius**2:
                    image[i, j] = 1
                    
    elif shape == "square":
        start = center - radius
        end = center + radius
        image[start:end, start:end] = 1
        
    elif shape == "triangle":
        height = 2 * radius
        for i in range(size):
            for j in range(size):
                # Check if point is inside an equilateral triangle
                if (i >= center - height/2) and (i <= center + height/2) and \
                   (j >= center - radius) and (j <= center + radius) and \
                   (i + j >= 2*center - radius):
                    image[i, j] = 1
                    
    elif shape == "cross":
        thickness = radius // 2
        # Horizontal bar
        image[center-thickness:center+thickness, center-radius:center+radius] = 1
        # Vertical bar
        image[center-radius:center+radius, center-thickness:center+thickness] = 1
        
    return image

# Create and display target image
shape_type = "circle"  # Try different shapes: "circle", "square", "triangle", "cross"
target_image = create_target_image(size=50, shape=shape_type)

plt.figure()
plt.imshow(target_image, cmap='gray')
plt.title(f"Target Image: {shape_type.capitalize()}")
plt.colorbar()
plt.show()

## Genetic Algorithm Setup - Fitness and Individual Definition

Define a "Individual" represented by an array of pixels, and a fitness function that minimizes the difference from our target image

In [ ]:
# Clear any previous creator settings to avoid errors if re-running
if 'FitnessMin' in creator.__dict__:
    del creator.FitnessMin
    del creator.Individual

# Define fitness (minimize difference from target)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

# Initialize toolbox
toolbox = base.Toolbox()

def create_random_image(size=50):
    """Generate a random binary image"""
    return np.random.randint(0, 2, (size, size))

# Register creation functions
toolbox.register("individual", tools.initIterate, creator.Individual, create_random_image)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Generate and display a random individual
random_individual = toolbox.individual()
plt.figure()
plt.imshow(random_individual, cmap='gray')
plt.title("Example Random Individual (Starting Point)")
plt.colorbar()
plt.show()

## Define Fitness Function and Genetic Operators

In [ ]:
# Fitness function: Mean squared error between images
def evaluate_image(individual):
    """Calculate fitness as MSE between individual and target image"""
    mse = np.sum((individual - target_image) ** 2) / (individual.shape[0] * individual.shape[1])
    return mse,

# Register the evaluation function
toolbox.register("evaluate", evaluate_image)

# Custom crossover: Single point spatial crossover
def image_crossover(ind1, ind2):
    """
    Perform spatial crossover between two images.
    Randomly chooses a row to split the images and swaps parts.
    """
    size = ind1.shape[0]
    # Choose a row to split
    split_point = random.randint(1, size-1)
    
    # Create offspring (need to create proper Individual objects)
    offspring1 = creator.Individual(np.vstack((ind1[:split_point].copy(), ind2[split_point:].copy())))
    offspring2 = creator.Individual(np.vstack((ind2[:split_point].copy(), ind1[split_point:].copy())))
    
    return offspring1, offspring2

# Test crossover
parent1 = toolbox.individual()
parent2 = toolbox.individual()
child1, child2 = image_crossover(parent1, parent2)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes[0, 0].imshow(parent1, cmap='gray')
axes[0, 0].set_title("Parent 1")
axes[0, 1].imshow(parent2, cmap='gray')
axes[0, 1].set_title("Parent 2")
axes[1, 0].imshow(child1, cmap='gray')
axes[1, 0].set_title("Child 1")
axes[1, 1].imshow(child2, cmap='gray')
axes[1, 1].set_title("Child 2")
plt.tight_layout()
plt.show()

# Custom mutation: Flip pixels with a certain probability
def image_mutation(individual, indpb=0.01):
    """
    Mutate an image by randomly flipping pixels with probability indpb.
    
    Parameters:
    - individual: The image to mutate
    - indpb: Independent probability of each pixel being flipped
    """
    # Create a copy to avoid modifying the original
    result = individual.copy()
    
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if random.random() < indpb:
                result[i, j] = 1 - result[i, j]  # Flip the pixel
                
    # Return result properly wrapped as Individual
    return creator.Individual(result),

# Register genetic operators
toolbox.register("mate", image_crossover)
toolbox.register("mutate", image_mutation, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=3)

# Test mutation
original = toolbox.individual()
mutated, = image_mutation(original)  # Note: This returns a tuple, so we unpack it

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(original, cmap='gray')
axes[0].set_title("Original")
axes[1].imshow(mutated, cmap='gray')
axes[1].set_title("After Mutation")
plt.tight_layout()
plt.show()

## Define Visualization and Statistics Functions

In [ ]:
# Function to visualize the best individual from a population
def plot_best_individual(population, gen, show_fitness_history=False):
    """
    Plot the best individual from a population alongside the target image.
    
    Parameters:
    - population: Current population of individuals
    - gen: Current generation number
    - show_fitness_history: Whether to plot fitness history graph
    """
    best_ind = tools.selBest(population, 1)[0]
    fitness = best_ind.fitness.values[0]
    
    if show_fitness_history:
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
    else:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        
    plt.imshow(target_image, cmap='gray')
    plt.title("Target Image")
    plt.colorbar()
    
    if show_fitness_history:
        plt.subplot(1, 3, 2)
    else:
        plt.subplot(1, 2, 2)
        
    plt.imshow(best_ind, cmap='gray')
    plt.title(f"Generation {gen}, Fitness: {fitness:.4f}")
    plt.colorbar()
    
    if show_fitness_history and hasattr(plot_best_individual, 'fitness_history'):
        plt.subplot(1, 3, 3)
        plt.plot(plot_best_individual.fitness_history)
        plt.title("Best Fitness Over Generations")
        plt.xlabel("Generation")
        plt.ylabel("Fitness (lower is better)")
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Function to display a grid of individuals
def plot_population_sample(population, gen, sample_size=9):
    """
    Plot a sample of individuals from the population.
    
    Parameters:
    - population: Current population of individuals
    - gen: Current generation number
    - sample_size: Number of individuals to display
    """
    # Use top individuals for visualization
    individuals = tools.selBest(population, sample_size)
    
    # Calculate grid dimensions
    grid_size = int(np.ceil(np.sqrt(sample_size)))
    
    plt.figure(figsize=(12, 12))
    for i, ind in enumerate(individuals):
        if i >= sample_size:
            break
            
        plt.subplot(grid_size, grid_size, i+1)
        plt.imshow(ind, cmap='gray')
        plt.title(f"Fitness: {ind.fitness.values[0]:.4f}")
        plt.axis('off')
    
    plt.suptitle(f"Population Sample - Generation {gen}")
    plt.tight_layout()
    plt.show()

## Run the Genetic Algorithm

In [ ]:
# Initialize fitness history tracking
plot_best_individual.fitness_history = []

# Setup statistics to track
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Create initial population
population_size = 50
population = toolbox.population(n=population_size)

# Parameters
NGEN = 100  # Number of generations
CXPB = 0.7  # Crossover probability
MUTPB = 0.2  # Mutation probability

# Evaluate initial population
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

# Run the algorithm with progress tracking
start_time = time.time()

for gen in range(NGEN):
    # Select the next generation individuals
    offspring = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)
    
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    # Replace the current population with the offspring
    population = toolbox.select(offspring, k=len(population))
    
    # Record statistics
    record = stats.compile(population)
    print(f"Gen {gen}: Min={record['min']:.4f}, Avg={record['avg']:.4f}")
    
    # Track best fitness
    best_fitness = record['min']
    plot_best_individual.fitness_history.append(best_fitness)
    
    # Visualize progress periodically
    if gen % 10 == 0 or gen == NGEN - 1:
        plot_best_individual(population, gen, show_fitness_history=True)

# Display elapsed time
elapsed_time = time.time() - start_time
print(f"Evolution completed in {elapsed_time:.2f} seconds")

# Show final result
plot_best_individual(population, NGEN, show_fitness_history=True)

## Analyze the Evolution Process

In [ ]:
# Extract best individual
best_ind = tools.selBest(population, 1)[0]
best_fitness = best_ind.fitness.values[0]

# Calculate pixel-wise difference between best and target
difference = np.abs(best_ind - target_image)

# Display the comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(target_image, cmap='gray')
axes[0].set_title("Target Image")
axes[1].imshow(best_ind, cmap='gray')
axes[1].set_title(f"Best Solution\nFitness: {best_fitness:.4f}")
axes[2].imshow(difference, cmap='hot')
axes[2].set_title("Difference Map")

for ax in axes:
    ax.axis('off')
    
plt.tight_layout()
plt.show()

# Print statistics
accuracy = (1 - np.sum(difference) / difference.size) * 100
print(f"Final fitness: {best_fitness:.4f}")
print(f"Pixel accuracy: {accuracy:.2f}%")
print(f"Total generations: {NGEN}")
print(f"Population size: {population_size}")